2일차 수업의 마지막인 8교시 미니 프로젝트는 2일차 6교시에서 다룬
‘데이터 불균형(Data Imbalance)’ 문제 해결과 4~5교시의 ‘심화 평가
지표(Recall, AUC)’ 활용을 총정리하는 주제로 선정했습니다.

주제는 응급 의료 상황에서 매우 중요한 ‘뇌졸중(Stroke) 조기
예측’입니다. 뇌졸중은 발병률이 낮아(데이터 불균형) 찾기 어렵지만,
놓치면 치명적인(높은 재현율 요구) 전형적인 헬스케어 난제입니다.

------------------------------------------------------------------------

# 📘 \[8교시 미니 프로젝트\] 불균형 데이터 정복: 뇌졸중(Stroke) 조기 진단

## 1. 프로젝트 개요 및 데이터 정의

### 1.1. 프로젝트 목표

- 시나리오: 응급실에 내원한 환자들의 생체 신호 데이터를 분석하여
  뇌졸중 고위험군을 선별해야 한다. 하지만 전체 데이터 중 실제 뇌졸중
  환자는 5% 미만이다.
- 학습 목표:
  1.  극심한 데이터 불균형 상황을 시각화하여 확인한다.
  2.  SMOTE (Synthetic Minority Over-sampling Technique) 기법을
      사용하여 환자 데이터를 증강한다.
  3.  단순 정확도(Accuracy)가 아닌 F1-Score와 ROC-AUC로 모델
      성능을 평가한다.
  4.  SMOTE 적용 전후의 성능 차이를 비교하여 불균형 처리의 중요성을
      체감한다.

### 1.2. 사용할 데이터셋 (Synthetic Stroke Data)

Kaggle의 유명한 Stroke Prediction Dataset을 모티브로 한 가상 데이터를
생성한다.

| 변수명 | 설명 | 타입 | 예시 |
|:----------------:|:----------------:|:----------------:|:----------------:|
| age | 나이 | 수치형 | 67.0, 45.0 … |
| hypertension | 고혈압 여부 | 범주형 | 0, 1 |
| heart_disease | 심장질환 여부 | 범주형 | 0, 1 |
| avg_glucose_level | 평균 혈당 | 수치형 | 228.69, 105.0 … |
| bmi | 체질량지수 | 수치형 | 36.6, 28.1 … (결측 있음) |
| stroke | 뇌졸중 발생 여부 (Target) | 범주형 | 0 (95%), 1 (5%) |

------------------------------------------------------------------------

## 2. 프로젝트 개발 가이드 (Step-by-Step)

### Step 1. 데이터 탐색 및 전처리

- 데이터를 생성하고 `stroke` 컬럼의 클래스 비율(`value_counts`)을
  확인한다.
- `bmi` 컬럼에 결측치(NaN)가 포함되어 있다. 이를 평균값으로 채운다.
- 나이와 혈당 등의 수치형 변수를 스케일링한다.

### Step 2. 데이터 분할 (Split)

- 전체 데이터를 학습용(Train)과 테스트용(Test)으로 8:2로 분할한다.
- 주의: 불균형 데이터이므로 분할 시 클래스 비율을 유지하기 위해
  `stratify=y` 옵션을 사용한다.

### Step 3. 불균형 처리 (SMOTE) - 핵심 단계

- `imblearn.over_sampling`의 `SMOTE`를 사용하여 학습
  데이터(X_train)에만 오버 샘플링을 적용한다.
- 주의: 테스트 데이터(X_test)는 절대 건드리면 안 된다. (현실 세계의
  데이터는 조작하면 안 되기 때문)

### Step 4. 모델링 및 비교 평가

- Case 1: SMOTE를 적용하지 않은 원본 데이터로 `LogisticRegression`
  학습.
- Case 2: SMOTE로 증강된 데이터로 `LogisticRegression` 학습.
- 두 모델의 Classification Report와 ROC-AUC를 비교하여, SMOTE가
  재현율(Recall)을 얼마나 끌어올렸는지 확인한다.

------------------------------------------------------------------------

## 3. 구현 힌트 (Hints)

- 힌트 1 (Stratify): 데이터 분할 시 한쪽 클래스가 너무 적으면 테스트
  셋에 환자가 한 명도 안 들어갈 수 있다.

  ``` python
  train_test_split(..., stratify=y) # 원본 비율 유지하며 분할
  ```

- 힌트 2 (SMOTE 적용):

  ``` python
  smote = SMOTE(random_state=42)
  X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
  ```

- 힌트 3 (결측치): `fillna()` 함수를 사용하여 결측치를 채운 후
  모델에 넣어야 한다.

------------------------------------------------------------------------

## 4. 최종 구현 코드 및 결과

### 4.1. 데이터 생성 및 불균형 확인

``` python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# 1. 불균형 데이터 생성 (1000명 중 50명만 환자)
np.random.seed(42)
n_samples = 1000

age = np.random.randint(18, 85, n_samples)
hypertension = np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
heart_disease = np.random.choice([0, 1], n_samples, p=[0.95, 0.05])
avg_glucose_level = np.random.normal(100, 40, n_samples)
bmi = np.random.normal(28, 7, n_samples)
# 결측치 주입 (BMI에 약 5% NaN)
bmi[np.random.choice(n_samples, 50, replace=False)] = np.nan

# 뇌졸중 발생 로직 (나이, 고혈압, 혈당이 높으면 확률 증가)
logits = -6 + (0.05 * age) + (0.8 * hypertension) + (0.01 * avg_glucose_level)
probs = 1 / (1 + np.exp(-logits))
stroke = [1 if p > np.random.rand() else 0 for p in probs]

# 강제로 불균형 비율 조정 (약 5%만 1이 되도록)
stroke = np.array(stroke)
if sum(stroke) > 50:
    stroke[np.where(stroke==1)[0][50:]] = 0

df = pd.DataFrame({
    'age': age, 'hypertension': hypertension, 'heart_disease': heart_disease,
    'avg_glucose_level': avg_glucose_level, 'bmi': bmi, 'stroke': stroke
})

print("타겟(Stroke) 비율 확인:")
print(df['stroke'].value_counts(normalize=True))

sns.countplot(x='stroke', data=df)
plt.title("Imbalanced Target Distribution")
plt.show()
```

### 4.2. 전처리 (결측치, 분할, 스케일링)

``` python
# 1. 결측치 처리 (평균 대치)
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

# 2. X, y 분리
X = df.drop('stroke', axis=1)
y = df['stroke']

# 3. 데이터 분할 (Stratify 옵션 중요!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터 크기: {X_train_scaled.shape}")
print(f"학습 데이터 내 환자 수: {sum(y_train)}")
```

### 4.3. SMOTE 적용 및 모델링 비교

``` python
# --- Case 1: SMOTE 없이 학습 (Imbalanced) ---
model_raw = LogisticRegression(random_state=42)
model_raw.fit(X_train_scaled, y_train)
y_pred_raw = model_raw.predict(X_test_scaled)

# --- Case 2: SMOTE 적용 학습 (Balanced) ---
smote = SMOTE(random_state=42)
# 중요: X_train(학습 데이터)만 뻥튀기해야 함!
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_smote.shape}")
print(f"SMOTE 적용 후 환자 수: {sum(y_train_smote)} (정상인과 1:1 비율)")

model_smote = LogisticRegression(random_state=42)
model_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = model_smote.predict(X_test_scaled)
```

### 4.4. 결과 비교 및 해석

``` python
print("=== [Case 1] 일반 모델 (Imbalanced) ===")
print(classification_report(y_test, y_pred_raw))
print(f"ROC-AUC Score: {roc_auc_score(y_test, model_raw.predict_proba(X_test_scaled)[:, 1]):.4f}")

print("\n=== [Case 2] SMOTE 모델 (Balanced) ===")
print(classification_report(y_test, y_pred_smote))
print(f"ROC-AUC Score: {roc_auc_score(y_test, model_smote.predict_proba(X_test_scaled)[:, 1]):.4f}")

# Confusion Matrix 시각화 비교
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(confusion_matrix(y_test, y_pred_raw), annot=True, fmt='d', ax=axes[0], cmap='Blues')
axes[0].set_title("Case 1: Without SMOTE")

sns.heatmap(confusion_matrix(y_test, y_pred_smote), annot=True, fmt='d', ax=axes[1], cmap='Greens')
axes[1].set_title("Case 2: With SMOTE")

plt.show()

# [결과 해석 예시]
# Case 1 (일반 모델):
# - 정확도는 95%로 높지만, Confusion Matrix를 보면 실제 환자(1)를 거의 못 찾는다.
# - Recall(재현율)이 0.0 ~ 0.2 수준으로 매우 낮다. 위험한 모델이다.

# Case 2 (SMOTE 모델):
# - 정확도는 조금 떨어질 수 있지만, Recall(재현율)이 대폭 상승한다 (예: 0.7 ~ 0.8).
# - 실제 뇌졸중 환자를 찾아내는 능력이 훨씬 뛰어나므로, 응급실 선별 도구로 적합하다.
# - ROC-AUC 점수 또한 상승했음을 확인할 수 있다.
```